In [ ]:
import asyncio
import logging
import sys

from astropy.coordinates import Angle
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
#mtcs = MTCS(domain)
comcam = ComCam(domain)
mtptg = salobj.Remote(domain, "MTPtg")

#await asyncio.gather(mtcs.start_task, comcam.start_task)
await asyncio.gather(comcam.start_task, mtptg.start_task)

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'

In [ ]:
await comcam.take_bias(nbias=2, group_id=group_id)

In [ ]:
await comcam.take_darks(exptime=5.0, ndarks=2, group_id=group_id)

In [ ]:
mtptg.cmd_azElTarget.set(
    targetName="Flatfield position",
    azDegs=Angle(205.7, unit=u.deg).deg,
    elDegs=Angle(39.0, unit=u.deg).deg,
    rotPA=Angle(0.0, unit=u.deg).deg
)

In [ ]:
ack = await mtptg.cmd_azElTarget.start(timeout=30)
print(ack)

In [ ]:
await comcam.take_flats(exptime=2.0, nflats=2, group_id=group_id)

In [ ]:
#await comcam.take_spot(exptime=3.0, n=2, group_id=group_id)

In [ ]:
#await comcam.take_engtest(exptime=2.0, n=1, group_id=group_id, test_type="FE55", obs_note="Fake test")